In [35]:
from PubMedDownloader import PubMedEntrezDownloader
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()

downloader = PubMedEntrezDownloader(email="olandechris@gmail.com", api_key = os.getenv("PUBMED_API_KEY"))

pmids = downloader.advanced_search(
    #query="Diabetes treatment",
    #mesh_terms=["Music"],
    #title_words=,
    #journal = ,
    #publication_types=["Journal Article", "Review"],
    #languages=["eng"],
    date_from="2023/01/01",
    date_to = "2025/06/01",
    max_results=3000
)

articles = downloader.fetch_article_details(pmids)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
downloader.save_to_json(articles, f"research{timestamp}.json")

Searching PubMed with query:  AND 2023/01/01[PDAT]:2025/06/01[PDAT]
Found 3919060 total articles, retrieving 3000 IDs
Fetching batch 1/30 (100 articles)...
Fetching batch 2/30 (100 articles)...
Fetching batch 3/30 (100 articles)...
Fetching batch 4/30 (100 articles)...
Fetching batch 5/30 (100 articles)...
Fetching batch 6/30 (100 articles)...
Fetching batch 7/30 (100 articles)...
Fetching batch 8/30 (100 articles)...
Fetching batch 9/30 (100 articles)...
Fetching batch 10/30 (100 articles)...
Fetching batch 11/30 (100 articles)...
Fetching batch 12/30 (100 articles)...
Fetching batch 13/30 (100 articles)...
Fetching batch 14/30 (100 articles)...
Fetching batch 15/30 (100 articles)...
Fetching batch 16/30 (100 articles)...
Fetching batch 17/30 (100 articles)...
Fetching batch 18/30 (100 articles)...
Fetching batch 19/30 (100 articles)...
Fetching batch 20/30 (100 articles)...
Fetching batch 21/30 (100 articles)...
Fetching batch 22/30 (100 articles)...
Fetching batch 23/30 (100 article

In [40]:
def get_article_statistics(articles):
        """Generate basic statistics about downloaded articles"""
        if not articles:
            return {}
        
        df = pd.DataFrame(articles)
        
        stats = {
            'total_articles': len(articles),
            'articles_with_abstracts': len(df[df['abstract'].str.len() > 0]),
            'date_range': {
                'earliest': df['year'].min(),
                'latest': df['year'].max()
            },
            'top_journals': df['journal'].value_counts().head(10).to_dict(),
            'publication_types': df['publication_types'].value_counts().head(10).to_dict(),
            'articles_per_year': df['year'].value_counts().sort_index().to_dict()
        }
        
        return stats

# Monkey patch it onto your instance
import types
downloader.get_article_statistics = types.MethodType(get_article_statistics, downloader)


In [ ]:
get_article_statistics(articles)

{'total_articles': 2998,
 'articles_with_abstracts': 2452,
 'date_range': {'earliest': '', 'latest': '2025'},
 'top_journals': {'Scientific reports': 34,
  'PloS one': 33,
  'Methods in molecular biology (Clifton, N.J.)': 26,
  'International journal of biological macromolecules': 24,
  'Cureus': 20,
  'Optics letters': 18,
  'Advanced science (Weinheim, Baden-Wurttemberg, Germany)': 17,
  'Journal of the American Chemical Society': 17,
  'Small (Weinheim an der Bergstrasse, Germany)': 16,
  'The Journal of organic chemistry': 15},
 'publication_types': {'Journal Article': 2216,
  'Journal Article; Review': 266,
  'Letter': 115,
  'Editorial': 71,
  'Published Erratum': 56,
  'Case Reports; Journal Article': 54,
  'News': 22,
  'Journal Article; Retraction Notice': 20,
  'English Abstract; Journal Article': 18,
  'Journal Article; Case Reports': 13},
 'articles_per_year': {'': 12, '2024': 10, '2025': 2976}}